In [1]:
from __future__ import annotations

import glob
import os
import time

from stable_baselines3 import PPO
from stable_baselines3.ppo import CnnPolicy, MlpPolicy

from pettingzoo.test import api_test
from lib.briscola_env.briscola_env import BriscolaEnv

In [2]:
env = BriscolaEnv()

api_test(env, num_cycles=1_000_000)

Starting API test


/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:140: UserWarning: Observation is not a NumPy array
  warnings.warn("Observation is not a NumPy array")
/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:253: UserWarning: Agents are recommended to have numbered string names, like player_0
  warnings.warn(
/home/tim/src/briscola-py/.venv/lib/python3.11/site-packages/pettingzoo/test/api_test.py:259: UserWarning: We recommend agents to be named in the format <descriptor>_<number>, like "player_0"
  warnings.warn(


Exception: card not playable: (5, swords)